In [12]:
import pandas as pd
import re

In [13]:
# ==============================
# ✅ Función de normalización para SapBERT
# ==============================
def normalizar_nombres(nombre):
    if pd.isna(nombre):
        return nombre

    nombre = nombre.lower().strip()

    # Unificar mg / ml / g / %
    nombre = re.sub(r'(\d+)\s*(mg|ml|gr|g|%)', r'\1 \2', nombre)

    # Estandarizar formato “x 30” → “30 unidades”
    nombre = re.sub(r'x\s*(\d+)', r'\1 unidades', nombre)

    # Diccionario de formas y abreviaturas comunes
    replacements = {
        "comp": "comprimidos",
        "comp.": "comprimidos",
        "caps": "capsulas",
        "cap": "capsulas",
        "tab": "tabletas",
        "tabl": "tabletas",
        "amp": "ampollas",
        "inj": "inyectable",
        "eferv": "efervescente",
        "sr": "liberacion sostenida",
        "sl": "sublingual",
        "nf": "nueva formula",
        "duo": "dual"
    }

    for k, v in replacements.items():
        nombre = re.sub(rf'\b{k}\b', v, nombre)

    # Eliminar espacios múltiples
    nombre = re.sub(r'\s+', ' ', nombre)

    return nombre.strip()



In [14]:
# ==============================
# 1️⃣ Cargar dataset unificado
# ==============================
df = pd.read_csv("../output/dataset_farmacia_unificado.csv")
print(f"Dataset original: {df.shape[0]} filas x {df.shape[1]} columnas")

Dataset original: 171867 filas x 16 columnas


In [15]:
# ==============================
# 4️⃣ Filtrar valores válidos
# ==============================

if 'monto_recibido' in df.columns:
    df = df[df['monto_recibido'] >= 0]
if 'precio' in df.columns:
    df = df[df['precio'] > 0]
if 'cantidad' in df.columns:
    df = df[df['cantidad'] >= 0]



In [16]:
# ==============================
# 5️⃣ Crear columna de cantidad total
# ==============================
if 'fraccion' in df.columns:
    df['cantidad_total'] = df['cantidad'] + df['fraccion'].fillna(0)
    df = df.drop(columns=['cantidad', 'fraccion'])
else:
    df['cantidad_total'] = df['cantidad']



In [17]:
# ==============================
# 7️⃣ Filtrar outliers (precio, total, cantidad_total)
# ==============================
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

for col in ['monto_recibido', 'precio', 'cantidad_total']:
    if col in df.columns:
        df = remove_outliers(df, col)

print(f"Dataset después de limpiar outliers: {df.shape[0]} filas x {df.shape[1]} columnas")


Dataset después de limpiar outliers: 127555 filas x 15 columnas


In [18]:
# ==============================
# 8️⃣ Eliminar columnas ID (no analíticas)
# ==============================
id_cols = [
    'auto_detalle', 'venta', 'compra2',
    'compra', 'medicamento', 'auto_med', 'auto'
]
df = df.drop(columns=id_cols, errors='ignore')



In [19]:
# Renombrar auto-venta y auto
df = df.rename(columns={
    'auto_venta': 'id_venta',
    'monto_recibido': 'total_venta',
    'total': 'total_unitario',
    'precio': 'precio_unitario'
})

In [20]:
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')


In [21]:
# ==============================
# 9️⃣ Normalización del nombre (SapBERT-Friendly)
# ==============================
if 'nombre' in df.columns:
    print("\nNormalizando nombres para SapBERT…")
    df["nombre_normalizado"] = df["nombre"].apply(normalizar_nombres)
    print("✅ Normalización completada.")


Normalizando nombres para SapBERT…
✅ Normalización completada.


In [22]:
# ==============================
# 9️⃣ Verificación final
# ==============================
print("\nTipos de datos finales:")
print(df.dtypes)

print("\nNulos por columna:")
print(df.isnull().sum())

print("\nPrimeras filas:")
print(df.head())


Tipos de datos finales:
id_venta                       int64
fecha                 datetime64[ns]
total_venta                  float64
precio_unitario              float64
total_unitario               float64
precio_compra                float64
nombre                        object
cantidad_total               float64
nombre_normalizado            object
dtype: object

Nulos por columna:
id_venta              0
fecha                 0
total_venta           0
precio_unitario       0
total_unitario        0
precio_compra         0
nombre                0
cantidad_total        0
nombre_normalizado    0
dtype: int64

Primeras filas:
   id_venta      fecha  total_venta  precio_unitario  total_unitario  \
0         4 2019-01-07        470.0              9.5             9.5   
1         4 2019-01-07        470.0              2.5             5.0   
2         4 2019-01-07        470.0              2.5             5.0   
3         4 2019-01-07        470.0              4.5             4.5   
4 

In [30]:
# ==============================
# 1️⃣ Dataset agrupado por id_venta
# ==============================
df_agrupado = df.groupby('id_venta').agg({
    'fecha': 'first',
    'total_venta': 'first',
    'cantidad_total': 'sum',
}).reset_index()

# ==============================
# 3️⃣ Guardar ambos datasets
# ==============================
# Guardamos el dataset agrupado
output_path_agrupado = "../output/dataset_farmacia_agrupado.csv"
df_agrupado.to_csv(output_path_agrupado, index=False)
print(f"\nDataset agrupado por id_venta guardado como: {output_path_agrupado}")

# Guardamos el dataset separado (ya lo habías hecho previamente)
output_path_separado = "../output/dataset_farmacia_separado.csv"
df.to_csv(output_path_separado, index=False)
print(f"Dataset separado guardado como: {output_path_separado}")

# ==============================
# 4️⃣ Confirmar la forma de los datasets
# ==============================
print(f"Forma del dataset agrupado: {df_agrupado.shape[0]} filas x {df_agrupado.shape[1]} columnas")
print(f"Forma del dataset separado: {df.shape[0]} filas x {df.shape[1]} columnas")



Dataset agrupado por id_venta guardado como: ../output/dataset_farmacia_agrupado.csv
Dataset separado guardado como: ../output/dataset_farmacia_separado.csv
Forma del dataset agrupado: 14815 filas x 4 columnas
Forma del dataset separado: 127555 filas x 9 columnas
